In [1]:
# base code from here:
# https://blog.francium.tech/custom-object-training-and-detection-with-yolov3-darknet-and-opencv-41542f2ff44e
# added some modifications


import argparse
from IPython.display import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt

config = '/home/ubuntu/darknet/cfg/yolov3-custom.cfg'
weights = '/home/ubuntu/darknet/model_first/yolov3-custom_3000.weights'
names = '/home/ubuntu/darknet/data/cone.names'

CONF_THRESH, NMS_THRESH = 0, 0.5
colors= [[0, 0, 255]]
print(colors)


def eval(img, original=None):
    if original is None:
        original = img
        
    # Load the network
    net = cv2.dnn.readNetFromDarknet(config, weights)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    # Get the output layer from YOLO
    layers = net.getLayerNames()
    output_layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    height, width = img.shape[:2]

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)

    class_ids, confidences, b_boxes = [], [], []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = 0
            confidence = scores[class_id]
            if confidence > CONF_THRESH:
                print("confidence", confidence)
            if confidence > CONF_THRESH:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                b_boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))

    try:
    # Perform non maximum suppression for the bounding boxes to filter overlapping and low confident bounding boxes
        indices = cv2.dnn.NMSBoxes(b_boxes, confidences, CONF_THRESH, NMS_THRESH).flatten()#.tolist()
    except:
        indices = []
    # Draw the filtered bounding boxes with their class to the image
    classes = ['cone']
    

    
    for index in indices:
        x, y, w, h = b_boxes[index]
        cv2.rectangle(original, (x, y), (x + w, y + h), colors[0], 2)

        cv2.putText(original, str(round(confidences[index], 2)), (x + 5, y + 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, colors[0], 2)
    return original

    
def eval_img(image_file):
    img = cv2.imread(image_file)
    edges = cv2.Canny(img, 100, 100)
    edges = cv2.cvtColor(edges,cv2.COLOR_GRAY2RGB)
    eval_img = eval(edges, img)
    
    plt.subplot(121),plt.imshow(img, cmap = 'gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(eval_img,cmap = 'gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    plt.show()


[[0, 0, 255]]
